# 【动手学Paddle2.0系列】模型训练的N种姿势


最开始接触深度学习的时候，我的码力几乎等于0，所以在最初的时候，几乎都是使用封装好的高层API进行训练，自由度很低。


想通过这个教程，对paddle2.0中的各种开启训练的方式进行一个总结。让我们开始愉快的学（ban）习（zhuan）吧！

## 1 Paddle2.0 主要思想


![](https://ai-studio-static-online.cdn.bcebos.com/785d759b1761429c8d433c1ed6219a4e5e61f800846340069235bf793a4a8dcd)


那么，用框架来类比，飞桨框架基础API对应方法一，飞桨框架高层API对应方法二。使用基础API，我们可以随心所欲的搭建自己的深度学习模型，不会受到任何限制；而使用方法二，我们可以很快的实现模型，达到自己想要的效果，缺点是少了一些自主性。但是，与制作披萨不同的是，飞桨框架可以做到真正的”鱼与熊掌”可以兼得，我们在飞桨框架中实现了API的高低融合，使我们的开发者既可以享受到基础API的强大，又可以兼顾高层API的快捷。


### 1.1 数据加载

In [ ]:
from paddle.vision.transforms import Compose, Normalize
from paddle.vision.datasets import MNIST


# 数据预处理，这里用到了随机调整亮度、对比度和饱和度
transform = Compose([Normalize(mean=[127.5],
                               std=[127.5],
                               data_format='CHW')])


# 数据加载，在训练集上应用数据预处理的操作
train_dataset = MNIST(mode='train', transform=transform)
test_dataset = MNIST(mode='test')

Cache file /home/aistudio/.cache/paddle/dataset/mnist/train-images-idx3-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/mnist/train-images-idx3-ubyte.gz 
Begin to download

Download finished
Cache file /home/aistudio/.cache/paddle/dataset/mnist/train-labels-idx1-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/mnist/train-labels-idx1-ubyte.gz 
Begin to download
........
Download finished
Cache file /home/aistudio/.cache/paddle/dataset/mnist/t10k-images-idx3-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/mnist/t10k-images-idx3-ubyte.gz 
Begin to download

Download finished
Cache file /home/aistudio/.cache/paddle/dataset/mnist/t10k-labels-idx1-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/mnist/t10k-labels-idx1-ubyte.gz 
Begin to download
..
Download finished


### 1.2 加载模型

In [ ]:
import paddle
from paddle.vision.models import LeNet
import paddle.nn as nn

mnist = LeNet()
# # 模型封装，用Model类封装
# model = paddle.Model(mnist)

# # 模型配置：为模型训练做准备，设置优化器，损失函数和精度计算方式
# model.prepare(optimizer=paddle.optimizer.Adam(parameters=model.parameters()),
#               loss=nn.CrossEntropyLoss(),
#               metrics=paddle.metric.Accuracy())


## 2 模型训练的N种姿势

### 2.1 高层API训练

#### 2.1.1 方式一

In [ ]:

# 模型训练，
model.fit(train_dataset,
          epochs=5,
          batch_size=64,
          verbose=1)

# 模型评估，
# model.evaluate(test_dataset, verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous step.
Epoch 1/5
step 938/938 [==============================] - loss: 0.0308 - acc: 0.9332 - 10ms/step         
Epoch 2/5
step 938/938 [==============================] - loss: 0.0267 - acc: 0.9752 - 9ms/step         
Epoch 3/5
step 938/938 [==============================] - loss: 0.0065 - acc: 0.9812 - 9ms/step         
Epoch 4/5
step 938/938 [==============================] - loss: 0.0057 - acc: 0.9841 - 10ms/step         
Epoch 5/5
step 938/938 [==============================] - loss: 0.0144 - acc: 0.9854 - 9ms/step         


Exception in thread Thread-15:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 338, in _thread_loop
    six.reraise(*sys.exc_info())
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/six.py", line 703, in reraise
    raise value
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 308, in _thread_loop
    batch = self._dataset_fetcher.fetch(indices)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/fetcher.py", line 65, in fetch
    data = self.collate_fn(data)
  File "/opt/con

Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.


SystemError: (Fatal) Blocking queue is killed because the data reader raises an exception.
  [Hint: Expected killed_ != true, but received killed_:1 == true:1.] (at /paddle/paddle/fluid/operators/reader/blocking_queue.h:154)
  [Hint: If you need C++ stacktraces for debugging, please set `FLAGS_call_stack_level=2`.]


#### 2.1.2 方式二

In [ ]:
# 构建训练集数据加载器
train_loader = paddle.io.DataLoader(train_dataset, batch_size=64, shuffle=True)

# 构建测试集数据加载器
test_loader = paddle.io.DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
model.fit(train_loader,
        eval_data = test_loader,
        epochs=5,
        verbose=1,
        )


The loss value printed in the log is the current step, and the metric is the average value of previous step.
Epoch 1/5
step 938/938 [==============================] - loss: 0.3847 - acc: 0.9869 - 9ms/step         


Exception in thread Thread-17:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 338, in _thread_loop
    six.reraise(*sys.exc_info())
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/six.py", line 703, in reraise
    raise value
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 308, in _thread_loop
    batch = self._dataset_fetcher.fetch(indices)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/fetcher.py", line 65, in fetch
    data = self.collate_fn(data)
  File "/opt/con

Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.


SystemError: (Fatal) Blocking queue is killed because the data reader raises an exception.
  [Hint: Expected killed_ != true, but received killed_:1 == true:1.] (at /paddle/paddle/fluid/operators/reader/blocking_queue.h:154)
  [Hint: If you need C++ stacktraces for debugging, please set `FLAGS_call_stack_level=2`.]


In [ ]:
model.evaluate(test_dataset, verbose=1)

#### 2.1.3 方式三

In [ ]:

for epoch in range(5):
    for batch_id, data in enumerate(train_loader()):
        x_data = data[0]
        y_data = data[1]
        info = model.train_batch([x_data], [y_data])

        if batch_id % 10 == 0:
            print(info)
             
    for ebatch_id, edata in enumerate(test_loader()):
        x_data_v = edata[0]
        y_data_v = edata[1]
        info = model.eval_batch([x_data_v], [y_data_v])

        if ebatch_id % 10 == 0:
            print(info)

([array([0.00321424], dtype=float32)], [1.0])
([array([0.02877501], dtype=float32)], [0.984375])
([array([0.03653406], dtype=float32)], [0.984375])
([array([0.00271822], dtype=float32)], [1.0])
([array([0.02068332], dtype=float32)], [1.0])
([array([0.01921546], dtype=float32)], [1.0])
([array([0.08911473], dtype=float32)], [0.96875])
([array([0.00550561], dtype=float32)], [1.0])
([array([0.03885828], dtype=float32)], [0.984375])
([array([0.0128423], dtype=float32)], [1.0])
([array([0.00481672], dtype=float32)], [1.0])
([array([0.00461083], dtype=float32)], [1.0])
([array([0.04832631], dtype=float32)], [0.96875])
([array([0.00081611], dtype=float32)], [1.0])
([array([0.0527472], dtype=float32)], [0.984375])
([array([0.02105747], dtype=float32)], [0.984375])
([array([0.03141265], dtype=float32)], [0.984375])
([array([0.06591309], dtype=float32)], [0.953125])
([array([0.00373719], dtype=float32)], [1.0])
([array([0.01238431], dtype=float32)], [1.0])
([array([0.069947], dtype=float32)], [0

Exception in thread Thread-21:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 338, in _thread_loop
    six.reraise(*sys.exc_info())
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/six.py", line 703, in reraise
    raise value
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 308, in _thread_loop
    batch = self._dataset_fetcher.fetch(indices)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/fetcher.py", line 65, in fetch
    data = self.collate_fn(data)
  File "/opt/con

([array([0.00776773], dtype=float32)], [1.0])


SystemError: (Fatal) Blocking queue is killed because the data reader raises an exception.
  [Hint: Expected killed_ != true, but received killed_:1 == true:1.] (at /paddle/paddle/fluid/operators/reader/blocking_queue.h:154)
  [Hint: If you need C++ stacktraces for debugging, please set `FLAGS_call_stack_level=2`.]


In [ ]:
model.evaluate(test_dataset, verbose=1)

#### 2.1.4 方式四

In [6]:
import paddle.nn.functional as F


mnist.train()
epochs = 5
optim = paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters())
# 用Adam作为优化函数
for epoch in range(epochs):
    for batch_id, data in enumerate(train_loader()):
        x_data = data[0]
        y_data = data[1]
        print(y_data)
    #     predicts = mnist(x_data)
    #     loss = F.cross_entropy(predicts, y_data)
    #     # 计算损失
    #     acc = paddle.metric.accuracy(predicts, y_data, k=2)
    #     loss.backward()
    #     if batch_id % 5 == 0:
    #         print("epoch: {}, batch_id: {}, loss is: {}, acc is: {}".format(epoch, batch_id, loss.numpy(), acc.numpy()))
    #     optim.step()
    #     optim.clear_grad()
    # for batch_id, data in enumerate(test_loader()):
    #     x_data = data[0]
    #     y_data = data[1]
        
    #     predicts = mnist(x_data)
    #     loss = F.cross_entropy(predicts, y_data)
    #     # 计算损失
    #     acc = paddle.metric.accuracy(predicts, y_data, k=2)
    #     if batch_id % 5 == 0:
    #         print("epoch: {}, batch_id: {}, loss is: {}, acc is: {}".format(epoch, batch_id, loss.numpy(), acc.numpy()))

KeyboardInterrupt: 

## 总结


在上面的四种训练方式中，实际上每种方式都有其优缺点，完全使用高层API能够方便的开启训练。对于完全的新手用户，推荐使用方式一、方式二。对于已经熟悉Paddle2.0的用户推荐使用方式四。方式四可以更加灵活，并且能够灵活地加入混合精度训练，训练方式更加自由。

### 一点小小的宣传&下期预告


我会在下一个项目中介绍在Paddle2.0中使用混合精度训练，欢迎大家关注哦。


我目前在上海，感兴趣的领域包括模型压缩、小目标检测、嵌入式，欢迎交流关注。[来AI Studio互粉吧~等你哦~ ](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/228777)